In [22]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [23]:
import torch
from utils import *
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
os.makedirs('F:/Citra Batch 3/Yusdiansya Putra/Bismillah Benar/Ukuran 256x256/SRGAN/a-PyTorch-Tutorial-to-Super-Resolution-master/Hasil Training SRResNet/Training SRGAN/Generated Epoch 100 Bt 8/Generate Epoch 100 Bt 8 Vid6/', exist_ok=True)

In [25]:
data_folder = './'

out_srresnet = 'F:/Citra Batch 3/Yusdiansya Putra/Bismillah Benar/Ukuran 256x256/SRGAN/a-PyTorch-Tutorial-to-Super-Resolution-master/Hasil Training SRResNet/Training SRGAN/Generated Epoch 100 Bt 8/Generate Epoch 100 Bt 8 Vid6/'


In [26]:
# Model checkpoints
srgan_checkpoint = "./E100Bt8_srgan.pth.tar"
# srresnet_checkpoint = "./E1000Bt32_srresnet.pth.tar"


In [27]:
# Load models
# srresnet = torch.load(srresnet_checkpoint)['model'].to(device)
# srresnet.eval()
srgan_generator = torch.load(srgan_checkpoint)['generator'].to(device)
srgan_generator.eval()

Generator(
  (net): SRResNet(
    (conv_block1): ConvolutionalBlock(
      (conv_block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
        (1): PReLU(num_parameters=1)
      )
    )
    (residual_blocks): Sequential(
      (0): ResidualBlock(
        (conv_block1): ConvolutionalBlock(
          (conv_block): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): PReLU(num_parameters=1)
          )
        )
        (conv_block2): ConvolutionalBlock(
          (conv_block): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
      )
      (1): ResidualBlock(
        (conv_block1): ConvolutionalBlock(
          (conv_block): Seque

In [28]:
def visualize_sr(img, halve=False):
    """
    Visualizes the super-resolved images from the SRResNet and SRGAN for comparison with the bicubic-upsampled image
    and the original high-resolution (HR) image, as done in the paper.

    :param img: filepath of the HR iamge
    :param halve: halve each dimension of the HR image to make sure it's not greater than the dimensions of your screen?
                  For instance, for a 2160p HR image, the LR image will be of 540p (1080p/4) resolution. On a 1080p screen,
                  you will therefore be looking at a comparison between a 540p LR image and a 1080p SR/HR image because
                  your 1080p screen can only display the 2160p SR/HR image at a downsampled 1080p. This is only an
                  APPARENT rescaling of 2x.
                  If you want to reduce HR resolution by a different extent, modify accordingly.
    """
    # Load image, downsample to obtain low-res version
    hr_img = Image.open(img, mode="r")
    hr_img = hr_img.convert('RGB')
    
    if halve:
        hr_img = hr_img.resize((int(hr_img.width / 2), int(hr_img.height / 2)),
                               Image.LANCZOS)
        
    lr_img = hr_img.resize((int(hr_img.width/2), int(hr_img.height/2)),
                               Image.BICUBIC)

    # Bicubic Upsampling
    bicubic_img = lr_img.resize((hr_img.width, hr_img.height), Image.BICUBIC)

#     # Super-resolution (SR) with SRResNet
#     sr_img_srresnet = srresnet(convert_image(lr_img, source='pil', target='imagenet-norm').unsqueeze(0).to(device))
#     sr_img_srresnet = sr_img_srresnet.squeeze(0).cpu().detach()
#     sr_img_srresnet = convert_image(sr_img_srresnet, source='[-1, 1]', target='pil')

#     Super-resolution (SR) with SRGAN
    sr_img_srgan = srgan_generator(convert_image(lr_img, source='pil', target='imagenet-norm').unsqueeze(0).to(device))
    sr_img_srgan = sr_img_srgan.squeeze(0).cpu().detach()
    sr_img_srgan = convert_image(sr_img_srgan, source='[-1, 1]', target='pil')
        

    return sr_img_srgan
